## Titanic Linear Model (grid search)

In [1]:
%%datalabframework getfilename

In [2]:
import datalabframework as dlf
logger = dlf.log.initLogger(__name__, kafka_topic="datalab", kafka_servers="kafka:9092")

2017-04-30 14:13:57,753 - jovyan - __main__ - INFO - init - {'datalab': {'framework': '0.1'}, 'notebook': {'filename': 'lr_grid.ipynb', 'filepath': '/home/jovyan/work/notebooks/models'}, 'project': {'rootpath': '/home/jovyan/work/notebooks', 'main': 'main.ipynb'}}


In [3]:
# EXPORT

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [4]:
# EXPORT

def lr_model(df):
   
    lr = LogisticRegression()
    
    grid = ParamGridBuilder() \
             .addGrid(lr.maxIter, [1,10,50,150,200,500])\
             .addGrid(lr.regParam, [0.01, 0.05, 0.1,]).build()
    
    evaluator = BinaryClassificationEvaluator()

    cv = CrossValidator(estimator=lr, estimatorParamMaps=grid, evaluator=evaluator)
    model = cv.fit(df)

    return model

In [7]:
DATA_ROOT = '/home/jovyan/work/data'
d = {   
    'input_sample' : 1.0,
    'input_source' : DATA_ROOT + '/titanic/data/set/train/features.parquet',
    'output_model' : DATA_ROOT + '/titanic/models/lr_tune.model'
}
p = dlf.params.config_fromdict(d)

In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName(dlf.project.filename()).getOrCreate()
spark.version

'2.1.0'

In [9]:
df = spark.read.parquet(p.input_source)
df.printSchema()
df.show()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Title: string (nullable = true)

+-----------+--------+------+------+------------------+-----+-----+-------+--------+------+
|PassengerId|Survived|Pclass|   Sex|               Age|SibSp|Parch|   Fare|Embarked| Title|
+-----------+--------+------+------+------------------+-----+-----+-------+--------+------+
|          1|       0|     3|  male|              22.0|    1|    0|   7.25|       S|    Mr|
|          2|       1|     1|female|              38.0|    1|    0|71.2833|       C|   Mrs|
|          3|       1|     3|female|              26.0|    0|    0|  7.925|       S|  Miss|
|          4|       1|     1|female|              35.0|    

In [10]:
from features import features
df_features = features.feature_vector(df, 
                  'PassengerId', 
                  'Survived', 
                  ['Pclass', 
                   'Sex', 
                   'Age', 
                   'SibSp', 
                   'Parch',
                   'Fare', 
                   'Embarked', 
                   'Title'])

importing Jupyter notebook from /home/jovyan/work/notebooks/features/features.ipynb


In [11]:
model = lr_model(df_features)

In [12]:
# Now we can optionally save the fitted pipeline to disk
import shutil
shutil.rmtree(p.output_model)

model.bestModel.save(p.output_model)

In [13]:
df_features.show(n=5,truncate=False)

+-----------+----------------------------------------------------+-----+
|PassengerId|features                                            |label|
+-----------+----------------------------------------------------+-----+
|1          |(24,[0,1,2,3,5,6,8],[3.0,1.0,22.0,1.0,7.25,1.0,1.0])|0.0  |
|2          |(24,[0,2,3,5,7,10],[1.0,38.0,1.0,71.2833,1.0,1.0])  |1.0  |
|3          |(24,[0,2,5,6,9],[3.0,26.0,7.925,1.0,1.0])           |1.0  |
|4          |(24,[0,2,3,5,6,10],[1.0,35.0,1.0,53.1,1.0,1.0])     |1.0  |
|5          |(24,[0,1,2,5,6,8],[3.0,1.0,35.0,8.05,1.0,1.0])      |0.0  |
+-----------+----------------------------------------------------+-----+
only showing top 5 rows



In [14]:
prediction = model.transform(df_features)
prediction.show(5)

+-----------+--------------------+-----+--------------------+--------------------+----------+
|PassengerId|            features|label|       rawPrediction|         probability|prediction|
+-----------+--------------------+-----+--------------------+--------------------+----------+
|          1|(24,[0,1,2,3,5,6,...|  0.0|[2.25429810242301...|[0.90502063632020...|       0.0|
|          2|(24,[0,2,3,5,7,10...|  1.0|[-2.3482572308897...|[0.08720439673290...|       1.0|
|          3|(24,[0,2,5,6,9],[...|  1.0|[-0.4276992620055...|[0.39467586017300...|       1.0|
|          4|(24,[0,2,3,5,6,10...|  1.0|[-1.9225232535820...|[0.12758045488993...|       1.0|
|          5|(24,[0,1,2,5,6,8]...|  0.0|[2.11735179401167...|[0.89257827729473...|       0.0|
+-----------+--------------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



In [15]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()

metric = evaluator.evaluate(prediction)
metric_name = evaluator.getMetricName()

print("Evaluation {} : {}".format(metric_name, metric))

Evaluation areaUnderROC : 0.8766630449834364
